In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Dataset loading

In [15]:
import pandas as pd
train = pd.read_csv(r'/content/drive/My Drive/finaltrain.csv',encoding='cp1252')
test = pd.read_csv(r'/content/drive/My Drive/finaltest.csv',encoding='cp1252')
autest = pd.read_csv(r'/content/drive/My Drive/UNSW_NB15_testing-set.csv',encoding='cp1252')

todrop = ['srcip', 'sport', 'dstip', 'dsport','sloss', 'dloss','stcpb', 'dtcpb', 'trans_depth', 'Stime', 'Ltime','ct_flw_http_mthd',
        'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm',
        'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat', 'Label']

trainAttributes = train.drop(todrop, axis = 1)
testAttributes = test.drop(todrop, axis = 1)

trainLabel = train['Label']
testLabel = test['Label']

autestdrop = ['ï»¿id', 'rate', 'sloss', 'dloss','stcpb', 'dtcpb','trans_depth','ct_srv_src','ct_dst_ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
       'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm',
       'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat']


autest = autest.drop(autestdrop, axis = 1)
#Change column names of autest
autest.rename(columns={'dinpkt':'Dintpkt','djit':'Djit','dload':'Dload','dpkts':'Dpkts','label':'Label','sinpkt':'Sintpkt','sjit':'Sjit', 'sload':'Sload', 'spkts':'Spkts','dmean':'dmeansz','response_body_len':'res_bdy_len', 'smean':'smeansz'}, inplace=True)
autest = autest[(autest['proto']=='tcp') | (autest['proto']=='udp') | (autest['proto']=='ospf')]
autest = autest[(autest['service']=='ssh') | (autest['service']=='ftp-data ')  | (autest['service']=='ftp') | (autest['service']=='-') | (autest['service']=='dns') | (autest['service']=='smtp') | (autest['service']=='http') | (autest['service']=='radius') | (autest['service']=='pop3') ]
autest = autest[(autest['state'] == 'CON') | (autest['state'] == 'RST') | (autest['state'] == 'FIN') | (autest['state'] == 'ACC') | (autest['state'] == 'REQ') | (autest['state'] == 'INT')]


<ipython-input-15-1d08cb19af35>:2: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(r'/content/drive/My Drive/finaltrain.csv',encoding='cp1252')
<ipython-input-15-1d08cb19af35>:3: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv(r'/content/drive/My Drive/finaltest.csv',encoding='cp1252')


In [16]:
autest['proto'].replace("tcp", 0, inplace = True)
autest['proto'].replace("udp", 1, inplace = True)
autest['proto'].replace("ospf", 2, inplace = True)

autest['service'].replace("ssh", 0, inplace = True)
autest['service'].replace("ftp-data", 1, inplace = True)
autest['service'].replace("ftp", 2, inplace = True)
autest['service'].replace("-", 3, inplace = True)
autest['service'].replace("dns", 4, inplace = True)
autest['service'].replace("smtp", 5, inplace = True)
autest['service'].replace("http", 6, inplace = True)
autest['service'].replace("radius", 7, inplace = True)
autest['service'].replace("pop3", 8, inplace = True)

autest['state'].replace("CON", 0, inplace = True)
autest['state'].replace("RST", 1, inplace = True)
autest['state'].replace("FIN", 2, inplace = True)
autest['state'].replace("ACC", 3, inplace = True)
autest['state'].replace("REQ", 4, inplace = True)
autest['state'].replace("INT", 5, inplace = True)

for column in autest.columns:
    if column != 'Label':
        col_mean = sum(autest[column]) / len(autest[column])
        col_std = (sum((x - col_mean) ** 2 for x in autest[column]) / len(autest[column])) ** 0.5
        autest[column] = [(x - col_mean) / col_std for x in autest[column]]


autest = autest.drop_duplicates()
autestAttributes = autest.drop(['Label'], axis = 1)
autestAttributes = autestAttributes[trainAttributes.columns]
autestLabel = autest['Label']

In [18]:
trainAttributes = np.array(trainAttributes)
trainLabel = np.array(trainLabel)


In [19]:
print("trainAttributes data type:", trainAttributes.dtype)
print("trainLabel data type:", trainLabel.dtype)

trainAttributes data type: float64
trainLabel data type: int64


# LSTM

In [20]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense


# Define the LSTM model
model = Sequential()
model.add(Dense(64, input_shape=(trainAttributes.shape[1],), activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(trainAttributes, trainLabel, epochs=10, batch_size=32)

Epoch 1/10
2532/2532 [==============================] - 7s 2ms/step - loss: 0.0466 - accuracy: 0.9908
Epoch 2/10
2532/2532 [==============================] - 6s 2ms/step - loss: 0.0338 - accuracy: 0.9928
Epoch 3/10
2532/2532 [==============================] - 5s 2ms/step - loss: 0.0327 - accuracy: 0.9929
Epoch 4/10
2532/2532 [==============================] - 6s 3ms/step - loss: 0.0322 - accuracy: 0.9929
Epoch 5/10
2532/2532 [==============================] - 5s 2ms/step - loss: 0.0319 - accuracy: 0.9930
Epoch 6/10
2532/2532 [==============================] - 6s 2ms/step - loss: 0.0315 - accuracy: 0.9929
Epoch 7/10
2532/2532 [==============================] - 5s 2ms/step - loss: 0.0313 - accuracy: 0.9930
Epoch 8/10
2532/2532 [==============================] - 7s 3ms/step - loss: 0.0309 - accuracy: 0.9930
Epoch 9/10
2532/2532 [==============================] - 8s 3ms/step - loss: 0.0312 - accuracy: 0.9930
Epoch 10/10
2532/2532 [==============================] - 5s 2ms/step - loss: 0.030

In [21]:
model.save_weights("lstmweights.h5")

In [22]:
predictions = model.predict(testAttributes)
predictions = (predictions > 0.5).astype(int)

1085/1085 [==============================] - 2s 2ms/step


# LSTM Performance

In [23]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(testLabel, predictions)

print("Classification Report for LSTM:")
print(report)

Classification Report for LSTM:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     17358
           1       0.99      1.00      0.99     17358

    accuracy                           0.99     34716
   macro avg       0.99      0.99      0.99     34716
weighted avg       0.99      0.99      0.99     34716



In [24]:
predictions = model.predict(autestAttributes)
predictions = (predictions > 0.5).astype(int)

1396/1396 [==============================] - 5s 3ms/step


In [25]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(autestLabel, predictions)

print("Classification Report for LSTM using Author's test dataset:")
print(report)

Classification Report for LSTM using Author's test dataset:
              precision    recall  f1-score   support

           0       0.68      0.98      0.80     29673
           1       0.70      0.08      0.15     14973

    accuracy                           0.68     44646
   macro avg       0.69      0.53      0.48     44646
weighted avg       0.69      0.68      0.58     44646



# GRU

In [27]:
trainAttributes = trainAttributes.reshape(trainAttributes.shape[0], trainAttributes.shape[1], 1)

In [29]:
import numpy as np
from keras.models import Sequential
from keras.layers import GRU, Dense

# Define the GRU model
# Define the GRU model
model = Sequential()
model.add(GRU(64, input_shape=(trainAttributes.shape[1], trainAttributes.shape[2]), activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification


# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(trainAttributes, trainLabel, epochs=10, batch_size=32)

Epoch 1/10
2532/2532 [==============================] - 64s 23ms/step - loss: 0.0651 - accuracy: 0.9886
Epoch 2/10
2532/2532 [==============================] - 37s 14ms/step - loss: 0.0373 - accuracy: 0.9918
Epoch 3/10
2532/2532 [==============================] - 36s 14ms/step - loss: 0.0352 - accuracy: 0.9926
Epoch 4/10
2532/2532 [==============================] - 35s 14ms/step - loss: 0.0340 - accuracy: 0.9927
Epoch 5/10
2532/2532 [==============================] - 36s 14ms/step - loss: 0.0336 - accuracy: 0.9927
Epoch 6/10
2532/2532 [==============================] - 35s 14ms/step - loss: 0.0327 - accuracy: 0.9930
Epoch 7/10
2532/2532 [==============================] - 38s 15ms/step - loss: 0.0327 - accuracy: 0.9930
Epoch 8/10
2532/2532 [==============================] - 35s 14ms/step - loss: 0.0325 - accuracy: 0.9930
Epoch 9/10
2532/2532 [==============================] - 36s 14ms/step - loss: 0.0326 - accuracy: 0.9930
Epoch 10/10
2532/2532 [==============================] - 35s 14m

In [30]:
model.save_weights("gruweights.h5")

# GRU Performance

In [31]:
predictions = model.predict(testAttributes)
predictions = (predictions > 0.5).astype(int)

from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(testLabel, predictions)

print("Classification Report for LSTM:")
print(report)

1085/1085 [==============================] - 6s 5ms/step
Classification Report for LSTM:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     17358
           1       0.99      1.00      0.99     17358

    accuracy                           0.99     34716
   macro avg       0.99      0.99      0.99     34716
weighted avg       0.99      0.99      0.99     34716



In [33]:
predictions = model.predict(autestAttributes)
predictions = (predictions > 0.5).astype(int)

from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(autestLabel, predictions)

print("Classification Report for LSTM using Author's test dataset:")
print(report)

1396/1396 [==============================] - 7s 5ms/step
Classification Report for LSTM using Author's test dataset:
              precision    recall  f1-score   support

           0       0.65      0.91      0.76     29673
           1       0.14      0.03      0.05     14973

    accuracy                           0.62     44646
   macro avg       0.39      0.47      0.40     44646
weighted avg       0.48      0.62      0.52     44646

